In [ ]:
# Instalación de librerias
! pip install numpy pandas

In [1]:
# Importación de librerias
import pandas as pd
import numpy as np

# Adjust display options to show all columns
pd.set_option('display.max_columns', None)

In [2]:
original_file_name = "tlf_hist_20240527.csv.gz" #Taken from Prisma - Sent from teams
aws_file_name = "ba0ef46f-9c2e-4f68-997e-bd2aaee0895e.csv" #Taken from AWS Athena - Prisma Analytics

# Load Files
path = "C:/Users/santi/OneDrive/Escritorio/Consultorias/2. Prisma/"
file_original = path + '2. Archivos/' + original_file_name
file_to_compare = path + '3. Validacion archivos/' + aws_file_name

## The sort_values variable depend of each logic file, normally you can order by id transaccion
variable_to_sort = 'id_transaccion'

df_original_sorted  = pd.read_csv(file_original, sep=';', dtype=str).sort_values(by= variable_to_sort).reset_index(drop=True)
df_to_compare_sorted  = pd.read_csv(file_to_compare, sep=',', dtype=str).sort_values(by= variable_to_sort).reset_index(drop=True)

### See which variables have a different field


In [18]:
common_columns = df_original_sorted.columns.intersection(df_to_compare_sorted.columns)

# Compare common columns
differences = {}
for column in common_columns:
    diff = df_original_sorted[column].compare(df_to_compare_sorted[column])
    if not diff.empty:
        differences[column] = diff.rename(columns={"self": "df_original", "other": "df_aws"})

# List columns with differences
columns_with_differences = [col for col, diff in differences.items() if not diff.empty]

# Combine differences into a single DataFrame for display
if differences:
    combined_differences = pd.concat(differences, axis=1)     

# Reset index and start from 1
combined_differences.index =    combined_differences.index + 1


combined_differences

af_tra_hora_transaccion                          f_val_monto            \
               df_original                   df_aws df_original    df_aws   
1                 07:56:19  2024-05-28 07:56:19.000       15000  15000.00   
2                 19:34:01  2024-05-28 19:34:01.000           0      0.00   
3                 12:44:28  2024-05-28 12:44:28.000           0      0.00   
4                 11:56:52  2024-05-28 11:56:52.000        2000   2000.00   
5                 09:59:15  2024-05-28 09:59:15.000           0      0.00   
6                 22:55:21  2024-05-28 22:55:21.000           0      0.00   
7                 16:53:58  2024-05-28 16:53:58.000           0      0.00   
8                 11:40:44  2024-05-28 11:40:44.000           0      0.00   
9                 15:04:47  2024-05-28 15:04:47.000       16000  16000.00   
10                11:31:48  2024-05-28 11:31:48.000           0      0.00   

   f_val_monto_1           f_val_monto_2        f_val_monto_3            \
     df_original    df_aws   df_original df_aws   df_original    df_aws   
1          15000  15000.00           NaN    NaN           NaN       NaN   
2              0      0.00             0   0.00             0      0.00   
3              0      0.00             0   0.00             0      0.00   
4           2000   2000.00           NaN    NaN             0      0.00   
5              0      0.00           NaN    NaN           NaN       NaN   
6              0      0.00           NaN    NaN         50000  50000.00   
7              0      0.00           NaN    NaN         40000  40000.00   
8              0      0.00             0   0.00             0      0.00   
9          16000  16000.00           NaN    NaN             0      0.00   
10             0      0.00             0   0.00             0      0.00   

   f_val_surcharge        af_tra_tiempo_total        f_val_monto_sorteo  \
       df_original df_aws         df_original df_aws        df_original   
1                0   0.00                 NaN    NaN                  0   
2                0   0.00                 .05   0.05                  0   
3                0   0.00                 .05   0.05                  0   
4                0   0.00                 .48   0.48                  0   
5                0   0.00                 .12   0.12                  0   
6                0   0.00                 .07   0.07                  0   
7                0   0.00                 .15   0.15                  0   
8                0   0.00                 .05   0.05                  0   
9                0   0.00                 .26   0.26                  0   
10               0   0.00                 .05   0.05                  0   

          f_val_surcharge_diferido         
   df_aws              df_original df_aws  
1    0.00                        0   0.00  
2    0.00                        0   0.00  
3    0.00                        0   0.00  
4    0.00                        0   0.00  
5    0.00                        0   0.00  
6    0.00                        0   0.00  
7    0.00                        0   0.00  
8    0.00                        0   0.00  
9    0.00                        0   0.00  
10   0.00                        0   0.00

### See if there are differences between the columns of the files


In [4]:
# Identify columns in each DataFrame
columns_original = set(df_original_sorted.columns)
columns_to_compare = set(df_to_compare_sorted.columns)

# Columns in original file but not in the file to compare
columns_only_in_original = columns_original - columns_to_compare

# Columns in the file to compare but not in the original file
columns_only_in_to_compare = columns_to_compare - columns_original

# Prepare the results
columns_diff = {
    "Only in original file": list(columns_only_in_original),
    "Only in file to compare": list(columns_only_in_to_compare)
}

columns_diff_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in columns_diff.items()]))

columns_diff_df

,Only in original file,Only in file to compare
0,id_tie_fechaHora_cal,id_tar_tarjeta_hasheado
1,id_tar_tarjeta,id_tie_fechahora_cal
2,NaN,dt
3,NaN,id_tar_tarjeta_encriptado


### Validations

In [20]:
df_original_sorted[['af_tra_hora_transaccion']]


,af_tra_hora_transaccion
0,07:56:19
1,19:34:01
2,12:44:28
3,11:56:52
4,09:59:15
5,22:55:21
6,16:53:58
7,11:40:44
8,15:04:47
9,11:31:48


In [19]:
df_to_compare_sorted[['af_tra_hora_transaccion']]

,af_tra_hora_transaccion
0,2024-05-28 07:56:19.000
1,2024-05-28 19:34:01.000
2,2024-05-28 12:44:28.000
3,2024-05-28 11:56:52.000
4,2024-05-28 09:59:15.000
5,2024-05-28 22:55:21.000
6,2024-05-28 16:53:58.000
7,2024-05-28 11:40:44.000
8,2024-05-28 15:04:47.000
9,2024-05-28 11:31:48.000
